In [ ]:
!pip install wandb

In [20]:
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torch.nn as nn
import torch.optim as optim
import numpy as np
import wandb

In [ ]:
!wandb login 

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [22]:
def train_val_dataset(dataset, val_split=0.1):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    return dict({
        "train" : Subset(dataset, train_idx),
        "val" : Subset(dataset, val_idx)
    })

def createDataLoader():
    mean_values = (0.5, 0.5, 0.5)
    std_values = (0.5, 0.5, 0.5)
    op_size = 32
    transform_train = transforms.Compose([
        transforms.RandomResizedCrop(op_size),
        transforms.ToTensor(),
        transforms.Normalize(mean_values,std_values ),
        ])

    transform_horizontal = transforms.Compose([
        transforms.RandomResizedCrop(op_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        ])
    transform_vertical = transforms.Compose([
        transforms.RandomResizedCrop(op_size),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        ])
    transform_Invert= transforms.Compose([
        transforms.RandomResizedCrop(op_size),
        transforms.RandomInvert(),
        transforms.ToTensor(),
        ])

    dataset_url = "/content/drive/MyDrive/dl-assigment-2/inaturalist_12K/train"

    img_data = torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_train)
    img_data_hori= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_horizontal)
    img_data_vert= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_vertical)
    img_data_inve= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_Invert)
    img_data = img_data + img_data_inve + img_data_vert + img_data_hori

    img_data = train_val_dataset(img_data)


    X_train=img_data['train']
    X_Valid=img_data['val']


    trainloader = torch.utils.data.DataLoader(X_train, batch_size=128, shuffle=True)
    validationloader = torch.utils.data.DataLoader(X_Valid, batch_size=128, shuffle=False)


    dataiter = iter(trainloader)
    # images, labels = dataiter.next()
    return trainloader,validationloader


def accuracy(dataset_itr,model,norm_fact):
    total = 0
    pred_count = 0
    for dataset in dataset_itr:
        X,y=dataset

        X = X.to(device)
        y = y.to(device)

        pred = torch.max(model(X,norm_fact).data,1)[1]

        total+=y.size(0)
        pred_count+=(pred==y).sum().item()
        acc = (100*pred_count)/total
    return acc


In [35]:
class CnnModel(nn.Module):
    def __init__(self,is_batch_norm,filter_size,file_org,F,d,actication_fun):
        super(CnnModel, self).__init__()
        if is_batch_norm:
            self.cnn_model = nn.Sequential(
                nn.Conv2d(3,filter_size,F),
                nn.BatchNorm2d(int(filter_size)),
                actication_fun,
                nn.MaxPool2d(2,stride=1),

                nn.Conv2d(filter_size,int(filter_size*file_org),F),
                nn.BatchNorm2d(int(filter_size*(file_org**1))),
                actication_fun,
                nn.MaxPool2d(2,stride=1),

                nn.Conv2d(int(filter_size*file_org),int(filter_size*(file_org**2)),F),
                nn.BatchNorm2d(int(filter_size*(file_org**2))),
                actication_fun,
                nn.MaxPool2d(2,stride=1),

                nn.Conv2d(int(filter_size*(file_org**2)),int(filter_size*(file_org**3)),F),
                nn.BatchNorm2d(int(filter_size*(file_org**3))),
                actication_fun,
                nn.MaxPool2d(2,stride=2),

                nn.Conv2d(int(filter_size*(file_org**3)),int(filter_size*(file_org**4)),int(F)),
                nn.BatchNorm2d(int(filter_size*(file_org**4))),
                actication_fun,
                nn.MaxPool2d(2,stride=2),

            )
        else:
            self.cnn_model = nn.Sequential(
            nn.Conv2d(3,filter_size,F),
            actication_fun,
            nn.MaxPool2d(2,stride=1),

            nn.Conv2d(filter_size,int(filter_size*file_org),F),
            actication_fun,
            nn.MaxPool2d(2,stride=1),

            nn.Conv2d(int(filter_size*file_org),int(filter_size*(file_org**2)),F),
            actication_fun,
            nn.MaxPool2d(2,stride=1),

            nn.Conv2d(int(filter_size*(file_org**2)),int(filter_size*(file_org**3)),F),
            actication_fun,
            nn.MaxPool2d(2,stride=2),

            nn.Conv2d(int(filter_size*(file_org**3)),int(filter_size*(file_org**4)),int(F)),
            actication_fun,
            nn.MaxPool2d(2,stride=2),

            output_dim=x_dim-self.F+1,

            )


        x_dim=32
        y_dim=x_dim-F+1

        x_dim=y_dim-2
        y_dim=x_dim-F+1
        y_dim=y_dim-1

        x_dim=y_dim
        y_dim=x_dim-F+1
        y_dim=y_dim-1

        x_dim=y_dim
        y_dim=x_dim-F+1
        y_dim=int(y_dim/2)

        x_dim=y_dim
        y_dim=x_dim-F+1
        y_dim=y_dim/2
        y_dim=int(y_dim)

        self.fc_model = nn.Sequential(
           nn.Linear(int(y_dim*y_dim*filter_size*(file_org**4)),120),
           nn.ReLU(),
           nn.Dropout(d),
           nn.Linear(120,10)
        )
    def forward(self, x,batch_norm):
        x = self.cnn_model(x)
        return self.fc_model(x.view(x.size(0), -1))


def fit(batch_norm,k,file_org,F,max_epochs,batch_size,d,act_fun=nn.ReLU()):
    model = CnnModel(is_batch_norm = batch_norm,
                        filter_size = k,
                        file_org = file_org,
                        F = F,
                        d = d,
                        actication_fun = act_fun).to(device)
    opt = optim.Adam(model.parameters())
    loss_fn = nn.CrossEntropyLoss()
    train_loss,val_loss=[],[]

    n_iter=np.ceil(8999/batch_size)
    trainloader,validationloader = createDataLoader()
    epoch = 0
    while epoch<max_epochs:
        for key,data in enumerate(trainloader,0):
            X,y=data
            X = X.to(device)
            y = y.to(device)
            opt.zero_grad()

            outputs=model(X,batch_norm)
            loss=loss_fn(outputs,y)
            loss.backward()
            opt.step()

            del X,y,outputs
            torch.cuda.empty_cache()

            if(key%100==0):
                print(f"Iter No. : {key}/{n_iter} , loss: {round(loss.item(),2)} ")



        for key,data in enumerate(validationloader,0):
            X,y=data
            X = X.to(device)
            y = y.to(device)

            outputs = model(X,batch_norm)
            loss = loss_fn(outputs,y)

            del X
            del y
            del outputs
            torch.cuda.empty_cache()

        epoch+=1
    val_loss.append(loss.item())
    train_loss.append(loss.item())
    wandb.log({
        "Epoch":epoch,
        "Train loss":train_loss[epoch],
        "validation loss":val_loss[epoch],
        "Validation Acc":accuracy(validationloader,model,batch_norm),
    })

    print("Training_accuracy:%.2f" % (accuracy(trainloader,model,batch_norm)))
    print("Validation_accuracy:%.2f" % (accuracy(validationloader,model,batch_norm)))

In [ ]:
hyperparameter_defaults= {
             "batch_norm":0,
             "no_of_layers":16,
             "file_org":1,
             "Filter_size":3,
              "max_epochs":2,
              "batch_size":128,
              "drop_out":0.3
}

In [ ]:
wandb.init(project="DL_Assignment_2",
           config={
             "batch_norm":0,
             "no_of_layers":16,
             "file_org":1,
             "Filter_size":3,
              "max_epochs":2,
              "batch_size":128,
              "act_fun":nn.ReLU(),
              "drop_out":0.3
           }
           )

In [ ]:
sweep_config={
    "name":"Assignment_2_PART_A",
    "method":"grid",
    "metric":{
        'name':'accuracy',
        'goal' : 'maximize'
    },
    "parameters":{
      "batch_norm":{
          "values":[0]  
      },
      "no_of_layers":{
          "values":[16,32]
      },
      "file_org":{
          "values":[0.5,1,2]
      },
      "Filter_size":{
          "values":[3]
      },
      "max_epochs":{
          "values":[20] 
      },
      "batch_size":{
          "values":[128]
      },
      "drop_out":{
          "values":[0.2,0.5]
      }


    }
}

In [ ]:
sweep_id12=wandb.sweep(sweep_config,entity="cs23m008",project="DL_Assignment_2")

In [ ]:
# fit(batch_norm=1,k=128,file_org=1,F=3,max_epochs=20,batch_size=128,d=0.0,act_fun=nn.ReLU())
def train():
  wandb.init(config=hyperparameter_defaults)
  config=wandb.config
  fit(batch_norm = config.batch_norm,
      k = config.no_of_layers,
      file_org = config.file_org,
      F = config.Filter_size,
      max_epochs = config.max_epochs,
      batch_size = config.batch_size,
      d = config.drop_out,
      act_fun=nn.ReLU())
   #change


In [ ]:
wandb.agent(sweep_id12,train)

# Question 4

In [ ]:
def createTrainDataLoader():
    mean_values = (0.5, 0.5, 0.5)
    std_values = (0.5, 0.5, 0.5)
    op_size = 32
    transform_train = transforms.Compose([
        transforms.RandomResizedCrop(op_size),
        transforms.ToTensor(),
        transforms.Normalize(mean_values,std_values ),
        ])

    transform_horizontal = transforms.Compose([
        transforms.RandomResizedCrop(op_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        ])
    transform_vertical = transforms.Compose([
        transforms.RandomResizedCrop(op_size),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        ])
    transform_Invert= transforms.Compose([
        transforms.RandomResizedCrop(op_size),
        transforms.RandomInvert(),
        transforms.ToTensor(),
        ])

    dataset_url = "/content/drive/MyDrive/dl-assigment-2/inaturalist_12K/train"

    img_data = torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_train)
    img_data_hori= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_horizontal)
    img_data_vert= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_vertical)
    img_data_inve= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_Invert)
    img_data = img_data + img_data_inve + img_data_vert + img_data_hori

    img_data = train_val_dataset(img_data)


    X_train=img_data['train']
    X_Valid=img_data['val']

    # test dataset
    dataset_url = "/content/drive/MyDrive/dl-assigment-2/inaturalist_12K/val"

    test_img_data = torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_train)
    test_img_data_hori= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_horizontal)
    test_img_data_vert= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_vertical)
    test_img_data_inve= torchvision.datasets.ImageFolder(root= dataset_url,  transform=transform_Invert)
    test_img_data = test_img_data + test_img_data_inve + test_img_data_vert + test_img_data_hori



    trainloader = torch.utils.data.DataLoader(X_train, batch_size=128, shuffle=True)
    testloader = torch.utils.data.DataLoader(test_img_data, batch_size=128, shuffle=True)
    validationloader = torch.utils.data.DataLoader(X_Valid, batch_size=128, shuffle=False)


    dataiter = iter(trainloader)
    # images, labels = dataiter.next()
    return trainloader,testloader

def fit(batch_norm,k,file_org,F,max_epochs,batch_size,d,act_fun=nn.ReLU()):
    model = CnnModel(is_batch_norm = batch_norm,
                        filter_size = k,
                        file_org = file_org,
                        F = F,
                        d = d,
                        actication_fun = act_fun).to(device)
    opt = optim.Adam(model.parameters())
    loss_fn = nn.CrossEntropyLoss()
    train_loss,test_loss=[],[]

    n_iter=np.ceil(8999/batch_size)
    trainloader,testloader = createTrainDataLoader()
    epoch = 0
    while epoch<max_epochs:
        for key,data in enumerate(trainloader,0):
            X,y=data
            X = X.to(device)
            y = y.to(device)
            opt.zero_grad()

            outputs=model(X,batch_norm)
            loss=loss_fn(outputs,y)
            loss.backward()
            opt.step()

            del X,y,outputs
            torch.cuda.empty_cache()

            if(key%100==0):
                print(f"Iter No. : {key}/{n_iter} , loss: {round(loss.item(),2)} ")



        for key,data in enumerate(testloader,0):
            X,y=data
            X = X.to(device)
            y = y.to(device)

            outputs = model(X,batch_norm)
            loss = loss_fn(outputs,y)

            del X
            del y
            del outputs
            torch.cuda.empty_cache()

        epoch+=1
    test_loss.append(loss.item())
    train_loss.append(loss.item())
    wandb.log({
        "Epoch":epoch,
        "Train loss":train_loss[epoch],
        "Testing loss":test_loss[epoch],
        "Test Acc":accuracy(testloader,model,batch_norm),
    })

    print("Training_accuracy:%.2f" % (accuracy(trainloader,model,batch_norm)))
    print("Test_accuracy:%.2f" % (accuracy(testloader,model,batch_norm)))
    return model

In [ ]:
best_hyperparameter = {
             "batch_norm":0,
             "no_of_layers":16,
             "file_org":1,
             "Filter_size":3,
              "max_epochs":2,
              "batch_size":128,
              "drop_out":0.3
}
def train_with_best_params():
  wandb.init(config=best_hyperparameter)
  config=wandb.config
  return fit(batch_norm = config.batch_norm,
      k = config.no_of_layers,
      file_org = config.file_org,
      F = config.Filter_size,
      max_epochs = config.max_epochs,
      batch_size = config.batch_size,
      d = config.drop_out,
      act_fun=nn.ReLU())
best_model = train_with_best_params()


## Question-4 B

In [ ]:
transform_test = transforms.Compose([
  transforms.RandomResizedCrop(32), 
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
sample_classes = ('Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia')
testing_data= torchvision.datasets.ImageFolder(root= "/content/train_local/inaturalist_12K/val",  transform=transform_test)
testdata_loader = torch.utils.data.DataLoader(testing_data, batch_size=30, shuffle=True)
testdata_itr = iter(testdata_loader)
sample_iamges, sample_labels = testdata_itr.next()
out_test_c = best_model(sample_iamges,best_hyperparameter.batch_norm)
imag=sample_iamges[0].unsqueeze(0)
out_test_c = best_model(imag,best_hyperparameter.batch_norm)
npimg = np.transpose(sample_iamges[0], (1, 2, 0))

train_img,img_title = [],[]
for i in range(30):
  imag=sample_iamges[i].unsqueeze(0)
  out_test_c = best_model(imag,best_hyperparameter.batch_norm) # Apply Forward propogation to already trained model.h
  train_img.append(transforms.ToPILImage()(torch.squeeze(sample_iamges[i])).convert("RGB"))
  predicted_title = sample_classes[out_test_c.argmax(dim=1).item()]
  correct_title = sample_classes[sample_labels[i].item()]
  combine_title = str("Actual Label : ") + correct_title + "\n" + str("Predicted Label : ") + predicted_title
  img_title.append(combine_title)
wandb.init(project="DL_Assignment_2", entity="cs23m008")
wandb.log({"sample image and prediction from the test dataset": [wandb.Image(img, caption=lbl) for img,lbl in zip(train_img,img_title)]})